In [ ]:
class user():
    def printTable(self,tablename):
        FETCH = f"SELECT * FROM {tablename}";
        cursor.execute(FETCH)
        colList = [i[0] for i in cursor.description]
        rTable = PrettyTable(colList)
        for rows in cursor.fetchall():
            rTable.add_row(rows)
        print(rTable)
        
    def signup(self):
        while(1):
            uname = input("Enter Username: ")
            IS_VALID_USERNAME = f"SELECT username FROM [crent].[dbo].[user] where username = '{uname}'"
            if(len(cursor.execute(IS_VALID_USERNAME).fetchall())):
                print("User already exists, please enter another user name")
                continue
            break
        
        pwd = input("Enter Passoword: ")
        hashString = hashlib.sha256(pwd.encode()).hexdigest()
        fname = input("Enter First Name: ")
        lname = input("Enter Last Name: ")
        gender = input("Enter Gender: ")
        email = input("Enter Email: ")
        address = input("Enter Address: ")
        delrequest = 'F'
        INSERT_USER = f"INSERT into [crent].[dbo].[user] VALUES ('{uname}','{hashString}','{fname}','{lname}','{gender}','{email}','{address}','{delrequest}')"
        cursor.execute(INSERT_USER)
        conn.commit()
        print(f"USER: {fname} Registered Successfully")
        
    def login(self):
        uname = input("Enter Username or email: ")
        pwd = input("Enter Password: ")
        hashString = hashlib.sha256(pwd.encode()).hexdigest()
        CHECK_USERID = f"SELECT username,email,password FROM [crent].[dbo].[user] where username = '{uname}' or email = '{uname}'"
        if(cursor.execute(CHECK_USERID).fetchone()!=None and (cursor.execute(CHECK_USERID).fetchone()[2] == hashString and cursor.execute(CHECK_USERID).fetchone()[0] ==  uname)):
            while(1):
                USER_DATA = f"SELECT * from [crent].[dbo].[user] where username = '{uname}' or email = '{uname}'"
                user_data = cursor.execute(USER_DATA).fetchone()
                userTable = PrettyTable([f"Welcome: {uname}"])
                userTable.add_row(["Press 1 for View all Car"])
                userTable.add_row(["Press 2 Book a Car"])
                userTable.add_row(["Press 3 for Return the Car and Make Transaction"])
                userTable.add_row(["Press 4 for Show book history"])
                userTable.add_row(["Press 5 to Update User Address"])
                userTable.add_row(["Press 6 request Account Deletion"])
                userTable.add_row(["Press 7 for Sign out"])
                print(userTable)
                ch = input("Enter your Choice: ")
                match ch:
                    case "1":
                        self.printTable('cars')
                        
                    case "2":
                        car_id = input("Enter Car ID: ")
                        CAR_AVAILABILITY = f"SELECT availcars FROM cars where carid = '{car_id}'"
                        availcars = cursor.execute(CAR_AVAILABILITY).fetchone()[0];
                        if(availcars == 0 or availcars == None):
                            print(f"Car id: '{car_id}' is not available now. Please book another car")
                        CAR_DETAILS = f"SELECT * FROM cars where carid ='{car_id}'"
                        car_data = cursor.execute(CAR_DETAILS).fetchone()
                        booking_id = cursor.execute("SELECT * FROM bookid").fetchone()[0]
                        username = user_data[0]
                        fullname = user_data[2]+" "+user_data[3]
                        gender = user_data[4]
                        carid = car_data[0]
                        model = car_data[1]
                        speed = car_data[2]
                        cost = car_data[3]
                        rating = 0
                        tid = ""
                        bookdate = date.today()
                        returndate = date(1996, 12, 11)
                        total_amount = 0
                        CASSANDRA_INS = "INSERT INTO cascrent.bookinghistory (booking_id, username, fullname, gender, carid, model, speed, cost, rating, tid, bookdate, returndate, total_amount) values({},'{}','{}','{}','{}','{}',{},{},{},'{}','{}','{}',{})".format(booking_id, username, fullname, gender, carid, model, speed, cost, rating, tid, bookdate, returndate, total_amount)
                        session.execute(CASSANDRA_INS)
                        print(f"Your Booking for Car ID: {carid} is Successful: Your booking id is: {booking_id}, Please note it for future reference")
                        availcars -=1
                        UPDATE_CARS = f"UPDATE cars set availcars = '{availcars}' where carid ='{car_id}'"
                        cursor.execute(UPDATE_CARS)
                        conn.commit()
                        cursor.execute(f"UPDATE bookid set bookingid ={booking_id+1} where bookingid = {booking_id}")
                        conn.commit()
                        
                    case "3":
                        bookingid = int(input("Enter the booking ID: "))
                        rating_user = float(input("Enter Rating (1-5): "))
                        if(rating_user <= 0):
                            rating_user = 1
                        if(rating_user>=5):
                            rating_user = 5
                        returndate = date.today()
                        CAS_CAR_DETAILS = f"SELECT carid,username,bookdate,tid FROM cascrent.bookinghistory WHERE booking_id ={bookingid} ALLOW FILTERING;"
                        cas_car_data = session.execute(CAS_CAR_DETAILS)
                        if(len(cas_car_data.current_rows)>0):
                            tidx = cas_car_data[0][3]
                            if(tidx != ''):
                                print("Car is Already returned or booking id is incorrect")
                                continue
                            car_id = cas_car_data[0][0]
                            username = cas_car_data[0][1]
                            car_data = cursor.execute(f"SELECT * FROM cars where carid = {car_id}").fetchone()
                            availcars = car_data[4]
                            cost = car_data[3]

                            txn_id = str(cursor.execute("SELECT bookingid FROM bookid").fetchone()[0])+"XFGH"
                            username = cas_car_data[0][1]
                            trtime = date.today()
                            bookdate = str(cas_car_data[0][2]).split('-')
                            year = int(bookdate[0])
                            month = int(bookdate[1])
                            day = int(bookdate[2])
                            datediff = (returndate - date(year,month,day)).days
                            if(datediff == 0):
                                datediff = 1
                            total_amount = cost*datediff
                            choice = input(f"Your Tota amount is {total_amount}: Do you want to return the car now? Y?N: ")
                            if(choice == 'Y'):
                                card_no = input('Enter 12 digit Debit Card No: ')
                                pin = input('Enter pin: ')
                                rating_sum = session.execute(f"SELECT sum(rating) from cascrent.bookinghistory where carid = '{car_id}' and rating > 0 ALLOW FILTERING;")[0][0]
                                rating_num = session.execute(f"SELECT count(*) from cascrent.bookinghistory where carid = '{car_id}' and rating > 0 ALLOW FILTERING")[0][0]
                                avg_rating = (rating_sum+rating_user)/(rating_num+1)
                                CAS_UPDATE = f"UPDATE cascrent.bookinghistory SET tid = '{txn_id}',rating = {rating_user},total_amount = {total_amount},returndate = '{trtime}' where booking_id = {bookingid}"
                                session.execute(CAS_UPDATE)
                                TXN_INSERT = f"INSERT INTO [crent].[dbo].[transaction] VALUES('{txn_id}','{username}','{returndate}',{total_amount},'DONE','{card_no}')"
                                cursor.execute(TXN_INSERT)
                                conn.commit()
                                cursor.execute(f"UPDATE cars set rating = {avg_rating}, availcars = {availcars+1} where carid = {car_id}")
                                conn.commit()
                                print(f"Your Transaction is Successful: Please note the Transaction Id: {txn_id} for future reference")
                            else:
                                print("Car not returned yet")
                        else:
                            print(f"Incorrect Booking ID: {bookingid}")
                            
                    case '4':
                        username_h = input("Enter User Name: ")
                        CAS_USER_DETAILS = f"SELECT * FROM cascrent.bookinghistoryuser WHERE username = '{username_h}';"
                        cas_user_data = session.execute(CAS_USER_DETAILS)
                        if(len(cas_user_data.current_rows) > 0):
                            length = session.execute(f"SELECT count(*) FROM cascrent.bookinghistoryuser WHERE username = '{username_h}';")[0][0]
                            colList = [i for i in cas_user_data.column_names]
                            historyTable = PrettyTable(colList)
                            for i in range(0,length):
                                user_data = []
                                for j in cas_user_data[i]:
                                    user_data.append(j)
                                historyTable.add_row(user_data)
                            print(historyTable)
                        else:
                            print(f"Incorrect User ID: {username_h} or user has not booked any car")
                            
                    case '5':
                        new_address = input("Enter New Address: ")
                        cursor.execute(f"UPDATE [crent].[dbo].[user] set address = '{new_address}' where username = '{uname}'")
                        conn.commit()
                        print("Address is Updated Successfully:")
                        
                    case '6':
                        cursor.execute(f"UPDATE [crent].[dbo].[user] set delrequest = 'T' where username = '{uname}'")
                        conn.commit()
                        print("Delete request sent successfully:")
                        
                    case '7':
                        print("Signed out Successfully:")
                        break 
                        
                    case _:
                        print("Please Choose to any listed operation: ")
        else:
            print("Username or Password is Incorrect: ")
        